0 - Set working directory

In [3]:
import os

# Always run notebook from project root (~/hotels)
os.chdir(os.path.expanduser("~/hotels"))
print("Working directory set to:", os.getcwd())

Working directory set to: /home/sandra/hotels


1 - Imports & paths

In [4]:
import pandas as pd
import numpy as np

# Define paths
raw_path = "data/raw/historical/hbd/hotel_bookings.csv"
processed_path = "data/processed/historical/hotel_bookings_clean.csv"


2 - Load & preview

In [5]:
# Load the raw dataset
df = pd.read_csv(raw_path)

# Shape and first rows
print("Shape:", df.shape)
df.head()


Shape: (119390, 32)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


3 - Basic cleaning

In [6]:
# Drop exact duplicates
df = df.drop_duplicates()

# Replace "NULL" strings with real NaN
df = df.replace("NULL", np.nan)

# Children column sometimes has NaN → replace with 0
df["children"] = df["children"].fillna(0)

# Check remaining missing values (top 10 columns)
df.isna().sum().sort_values(ascending=False).head(10)


company                     82137
agent                       12193
country                       452
hotel                           0
arrival_date_month              0
arrival_date_week_number        0
lead_time                       0
is_canceled                     0
stays_in_weekend_nights         0
stays_in_week_nights            0
dtype: int64

4 - Save processed copy

In [7]:
# Save the cleaned dataset (baseline version)
df.to_csv(processed_path, index=False)
print(f"Saved processed dataset to {processed_path}")

Saved processed dataset to data/processed/historical/hotel_bookings_clean.csv


5 - Inspect remaining NaN values

In [8]:
# Inspect all columns with missing values
missing_summary = df.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
missing_summary


company    82137
agent      12193
country      452
dtype: int64

6 - Handle common missing cases

In [9]:
# Fill children/babies more safely
df["children"] = df["children"].fillna(0)
df["babies"] = df["babies"].fillna(0)

# Fill agent and company with 0 (means 'no agent' / 'no company')
df["agent"] = df["agent"].fillna(0).astype(int)
df["company"] = df["company"].fillna(0).astype(int)

# Recheck
df.isna().sum().sort_values(ascending=False).head(10)


country                      452
hotel                          0
lead_time                      0
is_canceled                    0
arrival_date_month             0
arrival_date_week_number       0
arrival_date_day_of_month      0
arrival_date_year              0
stays_in_weekend_nights        0
stays_in_week_nights           0
dtype: int64

7 - Remove outliers in adr (average daily rate)

In [10]:
# Sometimes adr (average daily rate) has extreme values (like 5000+)
print("ADR before cleaning:", df["adr"].describe())

# Remove rows where adr < 0 or adr > 1000 (unrealistic)
df = df[(df["adr"] >= 0) & (df["adr"] <= 1000)]

print("ADR after cleaning:", df["adr"].describe())


ADR before cleaning: count    87396.000000
mean       106.337246
std         55.013953
min         -6.380000
25%         72.000000
50%         98.100000
75%        134.000000
max       5400.000000
Name: adr, dtype: float64
ADR after cleaning: count    87394.000000
mean       106.277964
std         52.017324
min          0.000000
25%         72.000000
50%         98.100000
75%        134.000000
max        510.000000
Name: adr, dtype: float64


8 - Final save

In [11]:
# Save cleaned dataset again
df.to_csv(processed_path, index=False)
print(f"Final cleaned dataset saved to {processed_path} with shape {df.shape}")


Final cleaned dataset saved to data/processed/historical/hotel_bookings_clean.csv with shape (87394, 32)


9 - Reload the cleaned dataset (safety)

In [12]:
df = pd.read_csv(processed_path)
print(df.shape)
df.head(2)


(87394, 32)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,0,0,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,0,0,0,Transient,0.0,0,0,Check-Out,2015-07-01


10 - Core features

In [13]:
# 1) Stay length
df["stay_length"] = df["stays_in_week_nights"] + df["stays_in_weekend_nights"]

# 2) Total guests + family flag
df["total_guests"] = df[["adults","children","babies"]].sum(axis=1)
df["is_family"] = (df[["children","babies"]].sum(axis=1) > 0).astype(int)

# 3) ADR per guest (avoid div by zero)
df["adr_per_guest"] = df["adr"] / df["total_guests"].replace(0,1)

# 4) Non-refundable vs refundable (assumption: 'Non Refund' = non-refundable)
df["non_refundable_flag"] = (df["deposit_type"] == "Non Refund").astype(int)
df["refundable_flag"] = 1 - df["non_refundable_flag"]

# Quick sanity
df[["stay_length","total_guests","is_family","adr_per_guest","refundable_flag"]].head()


,stay_length,total_guests,is_family,adr_per_guest,refundable_flag
0,0,2.0,0,0.0,1
1,0,2.0,0,0.0,1
2,1,1.0,0,75.0,1
3,1,1.0,0,75.0,1
4,2,2.0,0,49.0,1


11 - Check-in date & calendar features

In [14]:
# Build a proper check-in date
month_map = {'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,
             'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}
df["arrival_month_num"] = df["arrival_date_month"].map(month_map).astype(int)

df["checkin_date"] = pd.to_datetime(
    dict(year=df["arrival_date_year"],
         month=df["arrival_month_num"],
         day=df["arrival_date_day_of_month"])
)

# Calendar-derived features
df["checkin_dow"] = df["checkin_date"].dt.dayofweek            # 0=Mon, 6=Sun
df["is_weekend_checkin"] = df["checkin_dow"].isin([4,5,6]).astype(int)  # Fri/Sat/Sun

# Simple season buckets (N. Hemisphere)
def season(m):
    if m in (12,1,2): return "winter"
    if m in (3,4,5):  return "spring"
    if m in (6,7,8):  return "summer"
    return "autumn"
df["season"] = df["arrival_month_num"].apply(season)

df[["checkin_date","checkin_dow","is_weekend_checkin","season"]].head()


,checkin_date,checkin_dow,is_weekend_checkin,season
0,2015-07-01,2,0,summer
1,2015-07-01,2,0,summer
2,2015-07-01,2,0,summer
3,2015-07-01,2,0,summer
4,2015-07-01,2,0,summer


12 - Lead-time buckets

In [15]:
# Discretize lead_time into interpretable buckets
bins = [-1, 7, 30, 90, 180, 365, 10000]
labels = ["0-7d","8-30d","31-90d","91-180d","181-365d",">365d"]
df["lead_time_bucket"] = pd.cut(df["lead_time"], bins=bins, labels=labels)

df["lead_time_bucket"].value_counts(dropna=False)


lead_time_bucket
31-90d      22743
0-7d        18304
91-180d     18243
8-30d       16340
181-365d    11199
>365d         565
Name: count, dtype: int64

13 - Previous behaviour ratios

In [16]:
# Cancellation ratio from history (avoid division by zero)
den = df["previous_cancellations"] + df["previous_bookings_not_canceled"]
df["prev_cancel_ratio"] = df["previous_cancellations"] / den.replace(0, np.nan)
df["prev_cancel_ratio"] = df["prev_cancel_ratio"].fillna(0)

df["prev_cancel_ratio"].describe()


count    87394.000000
mean         0.013732
std          0.111818
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: prev_cancel_ratio, dtype: float64

14 - Save feature set

In [17]:
features_path = "data/processed/historical/hotel_bookings_features.csv"
df.to_csv(features_path, index=False)
print(f"Feature-enriched dataset saved to {features_path}  |  shape={df.shape}")


Feature-enriched dataset saved to data/processed/historical/hotel_bookings_features.csv  |  shape=(87394, 45)
